In [1]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [2]:
slovakia_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/drakh/slovakia-gps-data/refs/heads/master/GeoJSON/epsg_4326/districts_epsg_4326.geojson"
)
slovakia_gdf.crs = 4326

In [3]:
slovakia_gdf["NM3"] = "District of " + slovakia_gdf["NM3"]

## Get Open Population data

In [4]:
population_df = pd.read_csv("data/pop.csv", delimiter=";", encoding="cp1250")

In [5]:
population_df = population_df.rename(columns={"Indicator": "NM3"})
population_df["NM3"] = population_df["NM3"].str.replace("\xa0", " ", regex=False)
population_df = population_df.replace({"District of Śaľa": "District of Šaľa"})

In [6]:
population_df[["2014", "2024"]] = population_df[["2014", "2024"]].astype(int)

In [7]:
population_df["DIFF"] = population_df["2024"] - population_df["2014"]
population_df["PERCENT"] = (
    (population_df["DIFF"] / population_df["2014"]) * 100
).round(decimals=1)

## Merge data and export

In [8]:
slovakia_pop_gdf = slovakia_gdf.merge(population_df, on="NM3", how="left")

In [9]:
slovakia_pop_gdf = slovakia_pop_gdf.to_crs(5514)
slovakia_pop_gdf.to_file("data/slovakia_pop.gpkg")

In [10]:
population_df["2014"].sum()

np.int64(5417750)

In [11]:
population_df["2024"].sum()

np.int64(5422284)